### Preprocess

In [ ]:
# Get datasets
import pandas as pd

filepath    = 'E:/Research/Datasets/WSO2/Healthscore_dataset'
nps         = pd.read_csv(filepath + '/NPS.csv')

In [ ]:
# change feature names for ease of use
header_map = {
    "How likely are you to recommend WSO2 to a friend_ or colleague on a scale from 0 to 10? [0 being not at all likely and 10 being extremely likely]":'likely_to_recomend',
    "How satisfied are you with the support given by the WSO2 team?":'satisfaction',
    "Which response best captures the main impact of our product?":'product_impact',
    "How responsive have we been to your questions or concerns about our products?":'responsiveness'
}

nps.rename(columns=header_map,inplace=True)

In [ ]:
#######  view dataset ##############

# print total null values in each column
print('-----Null value count:-----\n\n',nps.isna().sum())

# print unique values in each column
print('\n\n--------Unique Value COunt-----------\n\n',nps.nunique())

# drop the columns that have many null values
temp_d1 = nps[['Account Name', 'Account Manager Name', 'UserName',
        'UserID', 'ResponseID',  'timeStamp',
       'dateTime',  'country',  'completion',
       'likely_to_recomend',
       'satisfaction',
       'responsiveness',
       'product_impact',
       'Sales Region', 'Sub Region', 'Survey Campaign', 'Segment']]

temp_d1.isna().sum()

In [ ]:
# temp_d1.nunique()

In [ ]:
##### Filling missing values in Sales Region #####
temp_d2 = temp_d1

# create a map between Sub Region ---- Sales Region
Sub_Regions = set(temp_d2['Sub Region'].tolist())
Region_Map  = dict() 

for sub_r in Sub_Regions:
    sales_r                 = temp_d2[(temp_d2['Sub Region']==sub_r)]['Sales Region'].unique()
    if len(sales_r)==1:
        Region_Map[sub_r]   = sales_r[0]
    elif len(sales_r)>1:
        print(sales_r)
        print('Impossible to have more than one sales region for a sub region')
    else:Region_Map[sub_r]  = None

still_null_regions = ['NA - CENTRAL','NA - SOUTH','NA - WEST','NA - EAST']
for region in still_null_regions:
    Region_Map[region]      = 'NA'



# Filling missing values in Sales Region
temp_d2['Sales Region']     = temp_d2['Sales Region'].fillna(temp_d2['Sub Region'].map(Region_Map))


In [ ]:
# save Region_Map
import numpy as np
np.save('E:/Research/CHS_Repo/Region_Map.npy', Region_Map) 

In [ ]:
# check whether there is any multiple responses from same account name

surveys = []
encoded_surveys =[]
survey_dates = list(temp_d2['Survey Campaign'].unique())

# encode dataset
def custom_encoder(d1):
     # ordinal encoding on features
    encoder_map_1 = {"Excellent":5,"Good":4,"Okay":3,"Bad":2,"Terrible":1}
    encoder_map_2 = {"Excellent":4,"Good":3,"OK":2,"Slow":1}
    encoder_map_3 = {"Many of the above":9,"High Quality":8,"Scalable":7,"Value for Money":6,"Useful":5,"Reliable":4,"Secure":3,"Unique":2,"None of the above":1}

    # --- satisfaction ----
    d1['encoded_satisfaction'] = d1.satisfaction.map(encoder_map_1)
    d1 = d1.drop(['satisfaction'],axis=1)

    # --- responsiveness ---
    d1['encoded_responsiveness'] = d1.responsiveness.map(encoder_map_2)
    d1 = d1.drop(['responsiveness'],axis=1)

    # --- product_impact ----
    d1['encoded_product_impact'] = d1.product_impact.map(encoder_map_3)
    d1 = d1.drop(['product_impact'],axis=1)
    return d1

for date in survey_dates:
    d1 = temp_d2[(temp_d2['Survey Campaign'] == date)]
    surveys.append(d1)

    # print length of each datasets
    print('Length of dataset - ',len(d1))

    # check is there any  duplicate accounts
    print('Duplicated\n') if True in d1[['Account Name']].duplicated().values else None

    # encode surveys
    d2 = custom_encoder(d1)
    encoded_surveys.append(d2)


In [ ]:
# See duplicate record
temp_d2 = temp_d2.drop_duplicates()
print(temp_d2.duplicated().sum())

In [ ]:
# get the duplicated account name list
i=2022
count=0
duplicate_acoounts = dict()
for survey in surveys:
    temp = []
    for true_val,account in zip(survey[['Account Name']].duplicated(),survey['Account Name']):
        if true_val:
            temp.append(account)
            duplicate_acoounts[i] = temp

            print('ID : ',i,account)
            count+= 1
    i+=1

In [ ]:
# Number of dupclicated accounts
print(count)
print(duplicate_acoounts)

In [ ]:
# Now i need to measure consensus(agreement) between records of duplicate account name 
# Here I am using Krippendorff's alpha method

'''
Here in Fleiss Kappa and Krippendorff's alpha what we are doing is if we have multiple raters in our scenario multiple responders,
we measure the agreement between multiple responses.

'''



In [ ]:
# calculate the agreement of duplicate responses
import krippendorff as kd

Krippendorff_dic    = dict()    # dictionary to store Krippendorff's alpha values
index               = 2022

for survey in encoded_surveys:                                                                         # add survey no as key
    accs                = survey['Account Name']
    duplicated_acc_list = list(survey[accs.isin(accs[accs.duplicated()])]['Account Name'].to_list())   # get names of account names which are duplicated
    temp_dic            = dict()
    
    
    for account in set(duplicated_acc_list):
        df = survey[(survey['Account Name']==account)][['likely_to_recomend','encoded_satisfaction','encoded_responsiveness']]
        df = df.fillna(-1)

        if len(df)==0:
            continue
        
        table               = df.values.tolist()

        Krippendorff        = kd.alpha(table,level_of_measurement='ordinal')          # calculating Krippendorff's alpha
        temp_dic[account]   = Krippendorff                                       # add Krippendorff to tempary dict
    
    Krippendorff_dic[index] = temp_dic 
    index+=1                                             # assign Krippendorff alpha of accounts in each survey
        

In [ ]:
''' 
Here I am considering low agreement data records as outliers. Since I can not decide which record is the outlier from multiple responses
from single account, I am going to drop all the responses belongs to that account name.
'''

In [ ]:
# check unique Account Names in each survey
# get the propotion of duplicated accounts
tot = 0
for survey in encoded_surveys:
    tot = tot+survey[['Account Name']].nunique()
    print(survey[['Account Name']].duplicated().sum()*100/len(survey[['Account Name']])  , '%')

print(tot,'\n')


In [ ]:
####### lets check how many will left after aggregating multiple responses  #######

# calculate how many of Krippendorff alpha values have above 0.6 value
keys = Krippendorff_dic.keys()
for key in keys:
    count=0
    k_values = Krippendorff_dic[key].values()
    for k in k_values:
        if k>=0.6:count+=1
    print('Krippendorff alpha value percentage with equal or more than 0.4 : \n',count*100/len(k_values),'\nTotal values:',len(k_values),
          '\n','Count : ',count,'\n')

In [ ]:
''' 
If we remove low agreement records we might loose lots of data records
But here most of records have atleast average agreement between multiple responds
'''

In [ ]:
def fillingRegions(d2,Region_Map):
    d2['Sales Region'] = d2['Sales Region'].fillna(d2['Sub Region'].map(Region_Map))
    return d2

In [ ]:
keys                    = Krippendorff_dic.keys()
high_agreement_surveys  = []
for key,survey in zip(keys,encoded_surveys):
    k_values    = Krippendorff_dic[key].values()        # getting Krippendorff alpha values of each multi responses
    k_account   =    Krippendorff_dic[key].keys()       # getting country of each multi responses
    survey      = fillingRegions(survey,Region_Map)     # filling missing sales region
    for k,acc in zip(k_values,k_account):
        if k<0.6:
            duplicates      = survey[(survey['Account Name']==acc)]
            print(duplicates[['likely_to_recomend','encoded_satisfaction', 'encoded_responsiveness', 'encoded_product_impact']])       # print low agrrement data
            
            survey          = survey.drop(survey[(survey['Account Name']==acc)].index)           # drop low agreement response
    high_agreement_surveys.append(survey)


In [ ]:
# no.of columns left
c     =   0
for i in range(5):
    c = c + len(high_agreement_surveys[i])
c

In [ ]:
# concat the surveys 
merged_surveys     = high_agreement_surveys[0]

for id in range(1,len(high_agreement_surveys)):
    merged_surveys = pd.concat([merged_surveys,high_agreement_surveys[id]] , axis=0)

merged_surveys

In [ ]:
# null values in each column
merged_surveys.isna().sum()

In [ ]:
'''Lets manually inspect for agreement of the data records'''

In [ ]:
keys = Krippendorff_dic.keys()

for key,survey in zip(keys,encoded_surveys):
    k_values    = Krippendorff_dic[key].values()       # getting Krippendorff alpha values of each multi responses
    k_account   =    Krippendorff_dic[key].keys()     # getting country of each multi responses
    survey      = fillingRegions(survey,Region_Map)      # filling missing sales region
    for k,acc in zip(k_values,k_account):
        if k>=0.6:
            temp_survey = survey[(survey['Account Name']==acc)]         # get high agreement data
            print(temp_survey[['likely_to_recomend','encoded_satisfaction', 'encoded_responsiveness', 'encoded_product_impact','dateTime']])     



In [ ]:
# save the merged dataset
merged_surveys.to_csv('E:/Research/Datasets/WSO2/Preprocessed datasets/higher_agreement_data.csv')

In [ ]:
'''Lets check outliers. Even though it is hard to identify'''

In [ ]:
temp_df1 = merged_surveys

# Define function to filter records based on likelihood to recommend mode
def filter_records(group):
    mode    = group['likely_to_recomend'].mode()
    median  = group['likely_to_recomend'].median()
    if len(mode) == 1:  
        group['diff_to_mode'] = abs(group['likely_to_recomend'] - mode[0])
        return group[group['diff_to_mode'] <= 5]
    else:
        group['diff_to_mode'] = abs(group['likely_to_recomend'] - median)
        return group[group['diff_to_mode'] <= 5]


filtered_df = temp_df1.groupby('Account Name').apply(filter_records)       # Apply the function to each group
filtered_df = filtered_df.drop(columns=['diff_to_mode']).reset_index(drop=True)            # Remove extra columns

In [ ]:
# Lets look at the dataset after removing outliers
print("Misssing value count of each feature:\n \n",filtered_df.isna().sum(),"\n\nUnique value count of each feature:\n",  max(filtered_df.nunique()))

In [ ]:
# filtered_df.to_csv('E:\Research\Datasets\WSO2\Preprocessed datasets/filtereddata.csv')

In [ ]:
'''Here what we are doing is we fill the missing values with account wise mode.
But we are not going to fill missing values of accounts that hvae multiple modes. 
Here mode means column wise mode.'''

In [ ]:
# Filling missing values of encoded_product_impact,encoded_responsiveness,encoded_satisfaction with mode,median

temp_df = filtered_df

# Define function to fill missing values with mode, excluding multiple modes
def fill_missing(group):
    for col in group[['encoded_satisfaction', 'encoded_responsiveness',
       'encoded_product_impact']].columns:  
        mode_values = group[col].mode()
        median_values = group[col].median()
        
        if len(mode_values)     == 1:                               # Only fill if there's a single mode
            group[col]      = group[col].fillna(mode_values[0])
        elif len(mode_values)   ==2 and col!='encoded_product_impact':
            group[col]      = group[col].fillna(mode_values.mean())
        elif len(mode_values)   >=3 and col!='encoded_product_impact':
            group[col]      = group[col].fillna(mode_values.median())
        else:
            if col!='encoded_product_impact':
                group[col] = group[col].fillna(median_values)
                
    
    return group

# Apply the function to each group
filled_df = temp_df.groupby('Account Name').apply(fill_missing).reset_index(drop=True)

In [ ]:
# Lets look at the dataset after filling missing values
print("Misssing value count of each feature:\n \n",filled_df.isna().sum(),"\n\nUnique value count of each feature:\n",  max(filled_df.nunique()))

In [ ]:
# Remove records with missing values
Cleaned_df = filled_df.dropna(subset=['encoded_responsiveness','encoded_satisfaction'])
Cleaned_df.nunique()

In [ ]:
# filtered_df.to_csv('E:/Research/Datasets/WSO2/Preprocessed datasets/filleddata.csv')
# filtered_df.to_csv('E:/Research/Datasets/WSO2/Preprocessed datasets/cleaneddata.csv')

In [ ]:
# '''ANother method is to handle missing value is dropping all the records with missing values'''

In [ ]:
# # let drop all the missing values
# temp_d3 = temp_d2.dropna()
# temp_d3.nunique()

### Labeling

In [ ]:
# Get the datasets
Cleaned_df = Cleaned_df
Filled_df = filled_df.fillna(-1)

In [ ]:
''' 
I can do labeling in two ways.
    1. I can use cleaned dataset and get a weighted score as a label
    2. I can use filled dataset which has null values and implement a method to labeling with null values
        - I use three fields for labeling (likely to recommend us, responsiveness , satisfaction of our service)
        - Only 2 contains null values (responsiveness , satisfaction of our service)


        (let assume all three has scaled to 100,10,10 respectively)
        Function to calculate healthscore label
            - healthScore = likeley to recommend - ((responsiveness.max - responsiveness) + (satisfaction of our service.max - satisfaction of our service))
'''

In [ ]:
######### calculate the label   ############

labelFields     = Filled_df[['likely_to_recomend','encoded_satisfaction', 'encoded_responsiveness']]        # Get required fields
weights         = [100,10,10]

# Scale the field values
for col,w in zip(labelFields.columns,weights):
    labelFields[col] =  labelFields[col]*w/(labelFields[col].max() - labelFields[col].min())


likely_to_recomend  =   labelFields['likely_to_recomend']
satisfaction        =   labelFields['encoded_satisfaction']
responsiveness      =   labelFields['encoded_responsiveness']
Filled_df['healthScore'] = likely_to_recomend - ( (satisfaction.max()  -  satisfaction ) +( responsiveness.max()  -  responsiveness) )

In [ ]:
import warnings
import itertools
import researchpy as rp
import matplotlib.pyplot as plt
import seaborn as sn

df                  = Filled_df.copy()
combinations        = list(itertools.combinations(df.dropna().columns, 2))
cramers_v_values    = pd.DataFrame(index=df.columns, columns=df.dropna().columns)

# Calculate Cramér's V for each pair
for feature1, feature2 in combinations:
    crosstab, results                           = rp.crosstab(df.dropna()[feature1], df.dropna()[feature2], test='chi-square')
    cramers_v_values.loc[feature1, feature2]    = results.loc[2, 'results']
    # print(f"Cramér's V for {feature1} and {feature2}: {results.loc[2, 'results']}")
    warnings.filterwarnings('ignore')


# Create a heatmap
cramers_v_values        = cramers_v_values.apply(pd.to_numeric)
plt.figure(figsize=(15,15))
sn.heatmap(cramers_v_values, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title("Cramér's V Heatmap")
plt.show()